In [1]:
import json
import sqlite3
from Definitions import doGet
from bs4 import BeautifulSoup

# We are creating a table, CompanyScrapingTable, baddna n7ott bi alba el: 
#     cik (cik)
#     cusip taba3 lcompany mn tall3o mnel filing (cusip)
#     el link taba3 awal filing 13 chi (first13)
#     lform (13G/A aw 13D/A aw 13D aw 13G) (form)
#     ltype taba3 lfiling, lli farjaytak yehoun mbere7 men 1 to 3 (type)
#     name of issuer, mn tall3o mnel filing (name of issuer)

conn = sqlite3.connect('Database.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS CompanyScrapingTable (
        cik TEXT PRIMARY KEY,
        cusip TEXT,
        first13 TEXT,
        form TEXT,
        type TEXT,
        ticker TEXT,
        nameOfIssuer TEXT
    )
''')

#3am njib kell el companies mnel object

cik_ticker_object = json.loads(doGet('https://www.sec.gov/files/company_tickers.json').content.decode('utf-8'))
ciks = []
tickers = []

generate = False

for company_index in cik_ticker_object:
    ciks.append(cik_ticker_object[company_index]["cik_str"])
    tickers.append(cik_ticker_object[company_index]["ticker"])

ciks = ciks
tickers = tickers


if generate: 
    try:
        #3am njib el JSON object yalli fi bi albo el company facts wl filings
        for cik in ciks:
            company_data = doGet(f"https://data.sec.gov/submissions/CIK{pad_string_with_zeros(str(cik))}.json").content.decode("utf-8")
            company_data = json.loads(company_data)
            #3am njib el accession number taba3 awal filing 13 chi 
            # el company_data['filings']['recent'] object taba3 el company byeje fiyo info aan kell lfilings aa he lchakel:
            # recent: {accessionNumber: [AN tb3 awwal filing, AN taba3 tene filing...], form: [form tb3 awal filing, form tb3 tene filing...]}
            #ne7na baddna njib el accession number tb3 awal filing 13 chi, la2an 3ayzino bl url tb3 lpage yalli mawjoud fiya el url tb3 lfiling. el afdaliye lal 13G/A
            filings = company_data.pop('filings')['recent']
            try:
                index_i = filings['form'].index('SC 13G/A')
                not_13GA = False
                form = '13G/A'
            except:
                not_13GA = True
                try:
                    print(f'D/A{cik}|{filings["form"].index("SC 13D/A")}')
                    index_i = filings['form'].index('SC 13D/A')
                    form = '13D/A'
                except:
                    try: 
                        print(f'G{cik}|{filings["form"].index("SC 13G")}')
                        index_i = filings['form'].index('SC 13D/A')
                        form = '13G'
                    except:
                        try:
                            print(f'D{cik}|{filings["form"].index("SC 13D")}')
                            index_i = filings['form'].index('SC 13D')
                            form = '13D'
                        except:                            
                            index_i = None
                            print(f'N{cik}')
                    
            if index_i != None:
                accession_number = filings['accessionNumber'][index_i]
                #halla2 hon 3am naamol scrape la page ta njib menna el url tb3 el filing yalli 3ayzino ta njib menno el cusip wl name_of_issuer
                response_str = doGet(f'https://www.sec.gov/Archives/edgar/data/{cik}/{"".join(accession_number.split("-"))}/{accession_number}-index.htm')  
                soup = BeautifulSoup(response_str.text, 'html.parser')
                filing_link_a_tag_index = 8
                filing_link = soup.find_all('a')[filing_link_a_tag_index]['href']
                while len(filing_link.split('.')) < 2:
                    filing_link_a_tag_index = filing_link_a_tag_index + 1    
                    filing_link = soup.find_all('a')[filing_link_a_tag_index]['href']
                filing_link = f'https://www.sec.gov{filing_link}' #
                print(filing_link)
                file = doGet(filing_link).text        
                #3am njib ltype taba3 lfiling (1, 2, aw 3)
                if filing_link.split(".")[3] == "txt":
                    type = 1
                else:
                    filing_13GA = BeautifulSoup(file, "html.parser")
                    cusip_tags = filing_13GA.find_all(lambda tag: "(cusip" in tag.get_text().lower())
                    cusip_number_tags = filing_13GA.find_all(lambda tag: "cusip number:" in tag.get_text().lower())
                    if len(cusip_tags) != 0:     
                        type = 2                   
                    else:
                        type = 4
              #3am ndif hal chabibe 3al database              
            cursor.execute('''INSERT OR REPLACE INTO CompanyScrapingTable (cik, type, ticker, first13, form) VALUES (?, ?, ?, ?) ''', (cik, type, tickers[ciks.index(cik)], filing_link, form))            
            conn.commit()
    except Exception as error:
        print(index_i)   
        print(filings['accessionNumber'])
        print(error)
# finally:
#     cursor.close()
#     conn.close() 






1
1045810
https://www.sec.gov/Archives/edgar/data/315066/000031506623001987/filing.txt
['67066G104\n', 'not_got', None]
731766
https://www.sec.gov/Archives/edgar/data/315066/000031506623002131/filing.txt
['91324P102\n', 'not_got', None]
1046179
https://www.sec.gov/Archives/edgar/data/1046179/000083423722010768/tw0002330008_100722.txt
['not got', 'not_got', None]
353278
https://www.sec.gov/Archives/edgar/data/353278/000130655023008717/dk0060534915_02323.txt
['not got', 'not_got', None]
884394
https://www.sec.gov/Archives/edgar/data/884394/000089542199000223/0000895421-99-000223.txt
['not got', 'not_got', None]
200406
https://www.sec.gov/Archives/edgar/data/93751/000009375123000419/Johnson_and_Johnson.txt
['not got', 'not_got', None]
937966
https://www.sec.gov/Archives/edgar/data/937966/000130655023008728/nl0010273215_022323.txt
['not got', 'not_got', None]
1551152
https://www.sec.gov/Archives/edgar/data/1364742/000130655023008705/us00287y1091_021523.txt
['not got', 'not_got', None]
1306

In [ ]:
#scraping el cusip wl name of issuer mnel 13 chi links la kell lcompanies and adding them to the database
import sqlite3
from bs4 import BeautifulSoup
from Definitions import doGet

conn = sqlite3.connect('Database.db')
cursor = conn.cursor()
cursor.execute('SELECT cik FROM CompanyScrapingTable')
ciks = cursor.fetchall()

def pad_string_with_zeros(input_string):
    num_zeros_to_add = 10 - len(input_string)
    if num_zeros_to_add > 0:
        padded_string = '0' * num_zeros_to_add + input_string
        return padded_string
    else:
        return input_string


def get_cusip_and_name_of_issuer(filing_link, cik):
                print(cik)
                filing_link = cursor.execute('SELECT first13 FROM CompanyScrapingTable WHERE cik = ?', (cik, )).fetchone()[0]
                form = cursor.execute('SELECT form FROM CompanyScrapingTable WHERE cik = ?', (cik, )).fetchone()[0]
                print(filing_link)
                file = doGet(filing_link).text 
                #if type 1 (txt file)       
                if filing_link.split(".")[3] == "txt":
                    type = 1
                    cusip = 'not_got'
                    name_of_issuer = 'not_got'
                    try:
                        cusip = "".join(file.split("(SEDOL")[0].split("(CUSIP")[0].split("Check")[0].split("-"))
                        if 'ecurities)' in cusip.lower():
                            cusip = re.split(r'(?:ECURITIES\)|ecurities\))', cusip)[1].split("-")[0]
                        if 'cusip #' in cusip.lower():
                            cusip = cusip.split('Cusip #')[1].split("-")[0]
                    except Exception as error:
                        cusip = 'not got'
                    try:
                        start_pattern = re.compile(r'item[^\w\d]*1[^\w\d]*a[^\w\d]*name[^\w\d]*of[^\w\d]*issuer[^\w\d]*', re.IGNORECASE)
                        end_pattern = re.compile(r'(item|\(B\)|-{2,})', re.IGNORECASE)
                        first = re.split(start_pattern, file)
                        name_of_issuer = re.split(end_pattern, re.split(start_pattern, file)[1])[0]
                    except Exception as error:
                        name_of_issuer = 'not_got'
                    if cusip == previous[0] or name_of_issuer == previous[1]:
                        print('duplicate')    
                    previous = [cusip, name_of_issuer]


                else:
                    filing_13GA = BeautifulSoup(file, "html.parser")
                    cusip_tags = filing_13GA.find_all(lambda tag: "(cusip" in tag.get_text().lower())
                    cusip_number_tags = filing_13GA.find_all(lambda tag: "cusip number:" in tag.get_text().lower())
                    #if type 2 (cusip present in tag above (CUSIP NUMBER) and name of issuer above (Name of issuer))
                    if len(cusip_tags) != 0:     
                        type = 2                   
                        for cusip_tag in cusip_tags:
                            previous_siblings = cusip_tag.find_previous_siblings()
                            for previous_sibling in previous_siblings:                
                                if previous_sibling.get_text(strip=True): 
                                    current_cusip_tag = cusip_tag
                                    extracted_text = previous_sibling.get_text()
                                    break
                        name_of_issuer_tags = filing_13GA.find_all(lambda tag: "(name" in tag.get_text().lower())                    
                        for name_of_issuer_tag in name_of_issuer_tags:
                            previous_siblings = name_of_issuer_tag.find_previous_siblings()
                            for previous_sibling in previous_siblings:                
                                if previous_sibling.get_text(strip=True): 
                                    current_name_of_issuer_tag = name_of_issuer_tag
                                    extracted_issuer = previous_sibling.get_text()
                                    break
                        name_of_issuer = extracted_issuer
                        cusip = extracted_text
                        if 'common' in extracted_text.lower():
                            if '(title of' in extracted_text.lower():
                                print('type 1 a')
                                name_of_issuer = current_name_of_issuer_tag.get_text().split('(N')[0]  
                                cusip = current_cusip_tag.get_text().split('(C')[0] #
                            elif ':' in extracted_text.lower():
                                print('type 1 B')
                                cusip = extracted_text.split(":")[1]
                        
                    #if type 3 (cusip present after cusip number: and name of issuer after name of issuer:)
                    elif len(cusip_number_tags) != 0:
                        type = 3
                        pattern = re.compile(r'Item\s*2\([a-zA-Z]\)\s*-\s*cusip\s*number', re.IGNORECASE)
                        cusip_number_element = filing_13GA.find('p', string=pattern)
                        if cusip_number_element:
                            next_p_tag = cusip_number_element.find_next('p')
                            cusip_number_element = next_p_tag.find('p', string=lambda text: pattern not in text)
                            if next_p_tag:
                                cusip = next_p_tag.get_text().strip()
                        pattern = re.compile(r'Item\s*1\([a-zA-Z]\)[\W_]*Name\s*of\s*Issuer', re.IGNORECASE)
                        name_of_issuer_element = filing_13GA.find('p', string=pattern)
                        if name_of_issuer_element:
                            next_p_tag = name_of_issuer_element.find_next('p')
                            issuer_name_element = next_p_tag.find('p', string=lambda text: pattern not in text)
                            if next_p_tag:
                                name_of_issuer = next_p_tag.get_text().strip()
                    else:
                        type = 4
                return [cusip, name_of_issuer, form]
            
conn = sqlite3.connect('Database.db')
cursor = conn.cursor()
cursor.execute('SELECT cik FROM CompanyScrapingTable')
ciks = cursor.fetchall()

#iterating over every type of 13X filing and getting the cusip and ticker
for type_value in range(1, 5):
    print(type_value)
    cursor.execute('SELECT * FROM CompanyScrapingTable WHERE type = ?', (type_value,))
    rows = cursor.fetchall()

    for row in rows:
        cik, _, filing_link, _, _,ticker, _ = row
        result = get_cusip_and_name_of_issuer(filing_link, cik)
        print(result)
        cursor.execute('''
            UPDATE CompanyScrapingTable
            SET cusip = ?, nameOfIssuer = ?, form = ?
            WHERE cik = ? AND type = ?
        ''', (result[0], result[1], result[2], cik, type_value, ))
        conn.commit()
